In [1]:
!pip install beautifulsoup4
!pip install aiohttp

  Using cached aiohttp-3.7.3-cp38-cp38-win_amd64.whl (634 kB)
  Using cached yarl-1.6.3-cp38-cp38-win_amd64.whl (125 kB)
  Using cached async_timeout-3.0.1-py3-none-any.whl (8.2 kB)
  Using cached multidict-5.0.2-cp38-cp38-win_amd64.whl (48 kB)


In [8]:
pip install tornado==4.5.3

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip show tornado

Name: tornado
Version: 4.5.3
Summary: Tornado is a Python web framework and asynchronous networking library, originally developed at FriendFeed.
Home-page: http://www.tornadoweb.org/
Author: Facebook
Author-email: python-tornado@googlegroups.com
License: http://www.apache.org/licenses/LICENSE-2.0
Location: c:\users\nehas\anaconda3\lib\site-packages
Requires: 
Required-by: terminado, notebook, jupyterlab, jupyter-client, ipykernel, distributed, bokeh, anaconda-project


In [7]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import aiohttp
import asyncio
import sys



async def get_issue_summary(issue_id, 
                              url_base = 'https://issues.apache.org/jira/browse'):
    """
    @param: user_id the unique user identifier in Benevity
    @param: url_base API base url
    @returns: Returns a dictionary of recommendations
    """
    recommendations_d = {}
    conn = aiohttp.TCPConnector(limit = 1, limit_per_host = 1)
    session = aiohttp.ClientSession(connector = conn)
    url = "/".join([url_base, str(issue_id)])
    print(f"Sending request to {url} ....")
    async with session.get(url) as resp:
        if resp.status == 200:
            response = await resp.text()
            if response:
                print(f"Received response from {url} ....")
                recommendations_d = response
            else:
                print(f"response not found for {url}!", sys.stderr)
        else:
            print(f"{url} ran into an error with status code: {resp.status} and response: {resp.text}", sys.stderr)
    await session.close()
    return recommendations_d # the dictionary of user and list of recommendations


async def get_issue_summaries(issue_list, issues_api, delay_per_request_seconds):
    """
    @param: user_list a python list of Benevity user ids to get remcommendations for.
    @returns: a dictionary of user_id_int to list of cause_in_int to recommend
    """
    # delay_per_request_seconds = 0.4 # @1000(0.35 for 8 processes, 0.16 for 3 p), @500(0.25 for 8 p)
    tasks = []
    for i in issue_list:
        tasks.append(asyncio.ensure_future(get_issue_summary(i, url_base = issues_api)))
        await asyncio.sleep(delay_per_request_seconds)
    results = await asyncio.gather(*tasks)
    return results 


def extract_summaries_from_html(list_of_html_objs):
    # id_list = list()
    summary_list = list()
    for html in list_of_html_objs:
        parsed_data = BeautifulSoup(html, 'html.parser')
        for row in parsed_data.find_all('title'):
            row_str = row.text
            print("title is: ", row_str)
            clean_row = row_str[row_str.find(']')+1 : row_str.find('- ASF')]
            # id_list.append(id)
            summary_list.append(clean_row)
#         print(summary_list)
    return summary_list


def generate_summary_csv(df):
    #prepare list of ids from df
    issue_id_list = df['ID'].tolist()
    url_base = 'https://issues.apache.org/jira/browse'
    asyncio.set_event_loop(asyncio.get_event_loop())
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(get_issue_summaries(issue_id_list,url_base,0.1))
    summary_list = extract_summaries_from_html(results)
        
    df_summary = pd.DataFrame(columns=["ID", "SUMMARY"])
    df_summary['ID'] = issue_id_list
    df_summary['SUMMARY'] = summary_list
    return df_summary
    



<font size="5">**Loading issue reports on which model will be trained**</font>

In [5]:
df_lucene = pd.read_csv (r'C:\Users\nehas\OneDrive\Documents\ENSF-612\Project\lucene_classification_vs_type.csv')
df_httpclient = pd.read_csv (r'C:\Users\nehas\OneDrive\Documents\ENSF-612\Project\httpclient_classification_vs_type.csv')
df_jackrabbit = pd.read_csv (r'C:\Users\nehas\OneDrive\Documents\ENSF-612\Project\jackrabbit_classification_vs_type.csv')



### Get summary for given issue Ids in above mentioned datasets

In [6]:
df_summary_lucene = generate_summary_csv(df_lucene)



Sending request to https://issues.apache.org/jira/browse/LUCENE-2719 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-754 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-1487 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2216 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-1061 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-456 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2243 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2458 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-754 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2670 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-3183 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2606 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-1487 ....
Received response from https://is

Sending request to https://issues.apache.org/jira/browse/LUCENE-3573 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-2444 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-1488 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2169 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-941 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-3776 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-1465 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-3658 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2796 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-1327 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2012 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-3074 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-3687 ....
Received re

Sending request to https://issues.apache.org/jira/browse/LUCENE-488 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-873 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-2067 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-1558 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2742 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-3556 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-2561 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-1427 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-1913 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2124 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-3249 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-2624 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-272 ....
Sending reque

Sending request to https://issues.apache.org/jira/browse/LUCENE-2512 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-3671 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-3239 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-444 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-3254 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-1718 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-450 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-1786 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-900 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-1444 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-1602 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-3589 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-1484 ....
Received response

Sending request to https://issues.apache.org/jira/browse/LUCENE-468 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-2037 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-2374 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-3820 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-1567 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-3470 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2860 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-2777 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-2701 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-229 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-2048 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-1329 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-651 ....
Sending reque

...

title is:  [LUCENE-475]  RAMDirectory(Directory dir, boolean closeDir)  constructor uses memory inefficiently. - ASF JIRA
title is:  [LUCENE-1868] update NOTICE.txt - ASF JIRA
title is:  [LUCENE-1137] Token type as BitSet: typeBits() - ASF JIRA
title is:  [LUCENE-3148] TestIndexWriterExceptions reproducible AOOBE in MockVariableIntBlockCodec - ASF JIRA
title is:  [LUCENE-2409] add a tokenfilter for icu transforms - ASF JIRA
title is:  [LUCENE-2474] Allow to plug in a Cache Eviction Listener to IndexReader to eagerly clean custom caches that use the IndexReader (getFieldCacheKey) - ASF JIRA
title is:  [LUCENE-1623] Back-compat break with non-ascii field names - ASF JIRA
title is:  [LUCENE-2977] WriteLineDocTask should write gzip/bzip2/txt according to the extension of specified output file name - ASF JIRA
title is:  [LUCENE-462] bad normalization in sorted search returning TopDocs - ASF JIRA
title is:  [LUCENE-3059] PulsingTermState.clone leaks memory - ASF JIRA
title is:  [LUCENE-2617]

title is:  [LUCENE-1066] better explain output - ASF JIRA
title is:  [LUCENE-2598] allow tests to use different Directory impls - ASF JIRA
title is:  [LUCENE-894] Custom build.xml for binary distributions - ASF JIRA
title is:  [LUCENE-3531] Improve CachingWrapperFilter to optionally also cache acceptDocs, if identical to liveDocs - ASF JIRA
title is:  [LUCENE-3905] BaseTokenStreamTestCase should test analyzers on real-ish content - ASF JIRA
title is:  [LUCENE-1347] IndexWriter.rollback can hang if a previous call hit an exception - ASF JIRA
title is:  [LUCENE-1211] Small speedups to DocumentsWriter's quickSort - ASF JIRA
title is:  [LUCENE-2142] FieldCache.getStringIndex should not throw exception if term count exceeds doc count - ASF JIRA
title is:  [LUCENE-2753] IndexReader.listCommits should return a List and not an abstract Collection - ASF JIRA
title is:  [LUCENE-3099] Grouping module should allow subclasses to set the group key per document - ASF JIRA
title is:  [LUCENE-3859] nuk

title is:  [LUCENE-3090] DWFlushControl does not take active DWPT out of the loop on fullFlush - ASF JIRA
title is:  [LUCENE-1162] Improve architecture of FieldSortedHitQueue - ASF JIRA
title is:  [LUCENE-3469] move DocumentStoredFieldsVisitor to o.a.l.document - ASF JIRA
title is:  [LUCENE-3579] DirectoryTaxonomyWriter should throw a proper exception if it was closed - ASF JIRA
title is:  [LUCENE-3097] Post grouping faceting - ASF JIRA
title is:  [LUCENE-2076] Add org.apache.lucene.store.FSDirectory.getDirectory() - ASF JIRA
title is:  [LUCENE-3412] SloppyPhraseScorer returns non-deterministic results for queries with many repeats - ASF JIRA
title is:  [LUCENE-1099] Making Tokenizer.reset(Reader) public - ASF JIRA
title is:  [LUCENE-1186] [PATCH] Clear ThreadLocal instances in close() - ASF JIRA
title is:  [LUCENE-917] Javadoc improvements for Payload class - ASF JIRA
title is:  [LUCENE-971] Create enwiki indexable data as line-per-article rather than file-per-article - ASF JIRA
title

title is:  [LUCENE-1675] Add a link to the release archive - ASF JIRA
title is:  [LUCENE-3572] MultiIndexDocValues pretends it can merge sorted sources - ASF JIRA
title is:  [LUCENE-2480] Remove support for pre-3.0 indexes - ASF JIRA
title is:  [LUCENE-1414] increase maxmemory for unit tests - ASF JIRA
title is:  [LUCENE-1158] DateTools UTC/GMT mismatch - ASF JIRA
title is:  [LUCENE-904] Calculate MD5 checksums in target <dist-all> - ASF JIRA
title is:  [LUCENE-3530] Remove deprecated methods in CompoundTokenFilters - ASF JIRA
title is:  [LUCENE-3019] FVH: uncontrollable color tags - ASF JIRA
title is:  [LUCENE-616] CLONE -Merge error during add to index (IndexOutOfBoundsException) - ASF JIRA
title is:  [LUCENE-1944] Remove deprecated Directory stuff and IR/IW open/ctor hell - ASF JIRA
title is:  [LUCENE-2924] fix getting started / demo docs - ASF JIRA
title is:  [LUCENE-1132] Highlighter Documentation updates - ASF JIRA
title is:  [LUCENE-2643] StringHelper#stringDifference is wrong a

title is:  [LUCENE-3771] Rename some remaining tests for new IndexReader class hierarchy - ASF JIRA
title is:  [LUCENE-1856] Remove Hits - ASF JIRA
title is:  [LUCENE-3723] Remove FieldMaskingSpanQuery (or fix its scoring) - ASF JIRA
title is:  [LUCENE-853] Caching does not work when using RMI - ASF JIRA
title is:  [LUCENE-2207] CJKTokenizer generates tokens with incorrect offsets - ASF JIRA
title is:  [LUCENE-1722] SmartChineseAnalyzer javadoc improvement - ASF JIRA
title is:  [LUCENE-3149] upgrade icu to 4.8 - ASF JIRA
title is:  [LUCENE-3724] SimpleText sumTotalTermFreq is wrong if only positions are omitted - ASF JIRA
title is:  [LUCENE-1015] FieldCache should support longs and doubles - ASF JIRA
title is:  [LUCENE-1864] bogus javadocs for FieldValueHitQuery.fillFields - ASF JIRA
title is:  [LUCENE-3386] Integrate MockBM25Similarity and MockLMSimilarity into the framework - ASF JIRA
title is:  [LUCENE-3250] remove contrib/misc and contrib/wordnet's dependencies on analyzers module 

In [ ]:
df_summary_httpclient = generate_summary_csv(df_httpclient)


In [ ]:
df_summary_jackrabbit = generate_summary_csv(df_jackrabbit)


### save the dataframe df_summary as a csv file.

In [ ]:
df_summary_lucene.to_csv('df_summary_lucene.csv')
df_summary_httpclient.to_csv('df_summary_httpclient.csv')
df_summary_jackrabbit.to_csv('df_summary_jackrabbit.csv')



In [ ]:
count_row = df_httpclient.shape[0]  # gives number of row count
count_col = df_httpclient.shape[1]  # gives number of col count

print(count_row)
print(count_col)

In [ ]:
count_row = df_summary_httpclient.shape[0]  # gives number of row count
count_col = df_summary_httpclient.shape[1]  # gives number of col count

print(count_row)
print(count_col)

## Get new data for given range of issue ids

In [4]:
def get_new_data_lucene():
    issue_id_list = list()
    lucene =  "LUCENE-"
    for id in range (5501, 5800):
#         print(id)
        lucene_id = lucene + str(id)
        issue_id_list.append(lucene_id)
    
    print(issue_id_list)
    url_base = 'https://issues.apache.org/jira/browse'
    asyncio.set_event_loop(asyncio.get_event_loop())
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(get_issue_summaries(issue_id_list,url_base,0.1))
    summary_list = extract_summaries_from_html(results)
        
    df_new_lucene_data = pd.DataFrame(columns=["ID", "SUMMARY"])
    df_new_lucene_data['ID'] = issue_id_list
    df_new_lucene_data['SUMMARY'] = summary_list
    df_new_lucene_data.to_csv('df_new_lucene_data.csv')
#     return df_new_lucene_data


def get_new_data_httpclient():
    issue_id_list = list()
    httpclient =  "HTTPCLIENT-"
    for id in range (1200, 1900):
#         print(id)
        httpclient_id = httpclient + str(id)
        issue_id_list.append(httpclient_id)
    
    print(issue_id_list)
    url_base = 'https://issues.apache.org/jira/browse'
    asyncio.set_event_loop(asyncio.get_event_loop())
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(get_issue_summaries(issue_id_list,url_base,0.1))
    summary_list = extract_summaries_from_html(results)
        
    df_new_httpclient_data = pd.DataFrame(columns=["ID", "SUMMARY"])
    df_new_httpclient_data['ID'] = issue_id_list
    df_new_httpclient_data['SUMMARY'] = summary_list
    df_new_httpclient_data.to_csv('df_new_httpclient_data_1.csv')
    

def get_new_data_jcr():
    issue_id_list = list()
    jcr =  "JCR-"
    for id in range (4420, 5000):
#         print(id)
        jcr_id = jcr + str(id)
        issue_id_list.append(jcr_id)
    
    print(issue_id_list)
    url_base = 'https://issues.apache.org/jira/browse'
    asyncio.set_event_loop(asyncio.get_event_loop())
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(get_issue_summaries(issue_id_list,url_base,0.1))
    summary_list = extract_summaries_from_html(results)
        
    df_new_jcr_data = pd.DataFrame(columns=["ID", "SUMMARY"])
    df_new_jcr_data['ID'] = issue_id_list
    df_new_jcr_data['SUMMARY'] = summary_list
    df_new_jcr_data.to_csv('df_new_jcr_data.csv')

In [5]:
get_new_data_lucene()

['LUCENE-5501', 'LUCENE-5502', 'LUCENE-5503', 'LUCENE-5504', 'LUCENE-5505', 'LUCENE-5506', 'LUCENE-5507', 'LUCENE-5508', 'LUCENE-5509', 'LUCENE-5510', 'LUCENE-5511', 'LUCENE-5512', 'LUCENE-5513', 'LUCENE-5514', 'LUCENE-5515', 'LUCENE-5516', 'LUCENE-5517', 'LUCENE-5518', 'LUCENE-5519', 'LUCENE-5520', 'LUCENE-5521', 'LUCENE-5522', 'LUCENE-5523', 'LUCENE-5524', 'LUCENE-5525', 'LUCENE-5526', 'LUCENE-5527', 'LUCENE-5528', 'LUCENE-5529', 'LUCENE-5530', 'LUCENE-5531', 'LUCENE-5532', 'LUCENE-5533', 'LUCENE-5534', 'LUCENE-5535', 'LUCENE-5536', 'LUCENE-5537', 'LUCENE-5538', 'LUCENE-5539', 'LUCENE-5540', 'LUCENE-5541', 'LUCENE-5542', 'LUCENE-5543', 'LUCENE-5544', 'LUCENE-5545', 'LUCENE-5546', 'LUCENE-5547', 'LUCENE-5548', 'LUCENE-5549', 'LUCENE-5550', 'LUCENE-5551', 'LUCENE-5552', 'LUCENE-5553', 'LUCENE-5554', 'LUCENE-5555', 'LUCENE-5556', 'LUCENE-5557', 'LUCENE-5558', 'LUCENE-5559', 'LUCENE-5560', 'LUCENE-5561', 'LUCENE-5562', 'LUCENE-5563', 'LUCENE-5564', 'LUCENE-5565', 'LUCENE-5566', 'LUCENE-5

Sending request to https://issues.apache.org/jira/browse/LUCENE-5530 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5523 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5524 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5531 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5532 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5525 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5533 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5526 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5534 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5527 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5535 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5528 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5536 ....
Sending reques

Sending request to https://issues.apache.org/jira/browse/LUCENE-5586 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5579 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5580 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5587 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5581 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5588 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5582 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5589 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5590 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5583 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5591 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5592 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5584 ....
Received respo

Sending request to https://issues.apache.org/jira/browse/LUCENE-5642 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5635 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5634 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5643 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5636 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5637 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5644 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5645 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5639 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5646 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5638 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5647 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5648 ....
Received respo

Sending request to https://issues.apache.org/jira/browse/LUCENE-5697 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5688 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5689 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5698 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5690 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5699 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5691 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5700 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5692 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5701 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5694 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5702 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5703 ....
Received respo

Sending request to https://issues.apache.org/jira/browse/LUCENE-5752 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5745 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5743 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5753 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5746 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5754 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5747 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5755 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5756 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5749 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5748 ....
Received response from https://issues.apache.org/jira/browse/LUCENE-5750 ....
Sending request to https://issues.apache.org/jira/browse/LUCENE-5757 ....
Received r

title is:  [LUCENE-5508] Unnecessary Check and Assgiment at FieldFacetStats - ASF JIRA
title is:  [LUCENE-5509] Equals Methods Does Not Consider Null Case - ASF JIRA
title is:  [LUCENE-5510] Docvalues need to be indexablefield (not storable) - ASF JIRA
title is:  [LUCENE-5511] Upgrade to SvnKit 1.8.4 for checks - ASF JIRA
title is:  [LUCENE-5512] Remove redundant typing (diamond operator) in trunk - ASF JIRA
title is:  [LUCENE-5513] Binary DocValues Updates - ASF JIRA
title is:  [LUCENE-5514] Backport Java 7 changes from trunk to Lucene 4.8 - ASF JIRA
title is:  [LUCENE-5515] Improve TopDocs#merge for pagination - ASF JIRA
title is:  [LUCENE-5516] Forward information that trigger a merge to MergeScheduler - ASF JIRA
title is:  [LUCENE-5517] stricter parsing for hunspell parseFlag() - ASF JIRA
title is:  [LUCENE-5518] minor hunspell optimizations - ASF JIRA
title is:  [LUCENE-5519] Make queueDepth enforcing optional in TopNSearcher - ASF JIRA
title is:  [LUCENE-5520] ArrayIndexOutOfBoun

title is:  [LUCENE-5611] Simplify the default indexing chain - ASF JIRA
title is:  [LUCENE-5612] LockStressTest fails always with NativeFSLockFactory - ASF JIRA
title is:  [LUCENE-5613] ByteBlockPool's documentation is completely useless - ASF JIRA
title is:  [LUCENE-5614] Add workaround for ANT bug to enable compile with Java 8 on ANT 1.8.3. and 1.8.4 - ASF JIRA
title is:  [LUCENE-5615] validate per-segment delete counts on write - ASF JIRA
title is:  [LUCENE-5616] remove FieldInfos leniency - ASF JIRA
title is:  [LUCENE-5617] don't rely on directory metadata for stored fields 'maxpointer', change to hard check - ASF JIRA
title is:  [LUCENE-5618] DocValues updates send wrong fieldinfos to codec producers - ASF JIRA
title is:  [LUCENE-5619] TestBackwardsCompatibility needs updatable docvalues - ASF JIRA
title is:  [LUCENE-5620] LowerCaseFilter.preserveOriginal - ASF JIRA
title is:  [LUCENE-5621] remove IndexOutput.flush() - ASF JIRA
title is:  [LUCENE-5622] Fail tests if they print ove

title is:  [LUCENE-5714] Improve tests for BBoxStrategy then port to 4x. - ASF JIRA
title is:  [LUCENE-5715] Upgrade direct dependencies known to be older than transitive dependencies - ASF JIRA
title is:  [LUCENE-5716] Track file handle leaks (FileDescriptor, NIO Path SPI and Socket mostly). - ASF JIRA
title is:  [LUCENE-5717] Postings highlighter support for multi term queries within filtered and constant score queries - ASF JIRA
title is:  [LUCENE-5718] More flexible compound queries (containing mtq) support in postings highlighter - ASF JIRA
title is:  [LUCENE-5719] Add additional checks and enforcement around Deprecation to the build system. - ASF JIRA
title is:  [LUCENE-5720] Optimize on disk packed integers part 2 - ASF JIRA
title is:  [LUCENE-5721] Monotonic packed could maybe be faster - ASF JIRA
title is:  [LUCENE-5722] Speed up MMapDirectory.seek() - ASF JIRA
title is:  [LUCENE-5723] Performance improvements for FastCharStream - ASF JIRA
title is:  [LUCENE-5724] CompoundFile

In [ ]:
get_new_data_httpclient()

In [3]:
get_new_data_jcr()

['JCR-4420', 'JCR-4421', 'JCR-4422', 'JCR-4423', 'JCR-4424', 'JCR-4425', 'JCR-4426', 'JCR-4427', 'JCR-4428', 'JCR-4429', 'JCR-4430', 'JCR-4431', 'JCR-4432', 'JCR-4433', 'JCR-4434', 'JCR-4435', 'JCR-4436', 'JCR-4437', 'JCR-4438', 'JCR-4439', 'JCR-4440', 'JCR-4441', 'JCR-4442', 'JCR-4443', 'JCR-4444', 'JCR-4445', 'JCR-4446', 'JCR-4447', 'JCR-4448', 'JCR-4449', 'JCR-4450', 'JCR-4451', 'JCR-4452', 'JCR-4453', 'JCR-4454', 'JCR-4455', 'JCR-4456', 'JCR-4457', 'JCR-4458', 'JCR-4459', 'JCR-4460', 'JCR-4461', 'JCR-4462', 'JCR-4463', 'JCR-4464', 'JCR-4465', 'JCR-4466', 'JCR-4467', 'JCR-4468', 'JCR-4469', 'JCR-4470', 'JCR-4471', 'JCR-4472', 'JCR-4473', 'JCR-4474', 'JCR-4475', 'JCR-4476', 'JCR-4477', 'JCR-4478', 'JCR-4479', 'JCR-4480', 'JCR-4481', 'JCR-4482', 'JCR-4483', 'JCR-4484', 'JCR-4485', 'JCR-4486', 'JCR-4487', 'JCR-4488', 'JCR-4489', 'JCR-4490', 'JCR-4491', 'JCR-4492', 'JCR-4493', 'JCR-4494', 'JCR-4495', 'JCR-4496', 'JCR-4497', 'JCR-4498', 'JCR-4499', 'JCR-4500', 'JCR-4501', 'JCR-4502', 'JC

Sending request to https://issues.apache.org/jira/browse/JCR-4434 ....
Received response from https://issues.apache.org/jira/browse/JCR-4427 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4435 ....
Received response from https://issues.apache.org/jira/browse/JCR-4428 ....
Received response from https://issues.apache.org/jira/browse/JCR-4429 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4436 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4437 ....
Received response from https://issues.apache.org/jira/browse/JCR-4430 ....
Received response from https://issues.apache.org/jira/browse/JCR-4431 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4438 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4439 ....
Received response from https://issues.apache.org/jira/browse/JCR-4432 ....
Received response from https://issues.apache.org/jira/browse/JCR-4433 ....
Sending request to https://issues.apache.org/jira

Sending request to https://issues.apache.org/jira/browse/JCR-4492 ....
Received response from https://issues.apache.org/jira/browse/JCR-4483 ....
Received response from https://issues.apache.org/jira/browse/JCR-4482 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4493 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4494 ....
Received response from https://issues.apache.org/jira/browse/JCR-4484 ....
Received response from https://issues.apache.org/jira/browse/JCR-4485 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4495 ....
Received response from https://issues.apache.org/jira/browse/JCR-4486 ....
Received response from https://issues.apache.org/jira/browse/JCR-4489 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4496 ....
Received response from https://issues.apache.org/jira/browse/JCR-4487 ....
Received response from https://issues.apache.org/jira/browse/JCR-4488 ....
Received response from https://issues.apache.

Sending request to https://issues.apache.org/jira/browse/JCR-4549 ....
Received response from https://issues.apache.org/jira/browse/JCR-4542 ....
Received response from https://issues.apache.org/jira/browse/JCR-4541 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4550 ....
Received response from https://issues.apache.org/jira/browse/JCR-4543 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4551 ....
Received response from https://issues.apache.org/jira/browse/JCR-4544 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4552 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4553 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4554 ....
Received response from https://issues.apache.org/jira/browse/JCR-4547 ....
Received response from https://issues.apache.org/jira/browse/JCR-4545 ....
Received response from https://issues.apache.org/jira/browse/JCR-4546 ....
Sending request to https://issues.apache.org/jira

Sending request to https://issues.apache.org/jira/browse/JCR-4606 ....
Received response from https://issues.apache.org/jira/browse/JCR-4600 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4607 ....
Received response from https://issues.apache.org/jira/browse/JCR-4601 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4608 ....
Received response from https://issues.apache.org/jira/browse/JCR-4602 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4609 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4610 ....
Received response from https://issues.apache.org/jira/browse/JCR-4603 ....
Received response from https://issues.apache.org/jira/browse/JCR-4604 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4611 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4612 ....
Received response from https://issues.apache.org/jira/browse/JCR-4605 ....
Received response from https://issues.apache.org/jira

...

Received response from https://issues.apache.org/jira/browse/JCR-4944 ....
Received response from https://issues.apache.org/jira/browse/JCR-4945 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4952 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4953 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4954 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4955 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4956 ....
Received response from https://issues.apache.org/jira/browse/JCR-4946 ....
Received response from https://issues.apache.org/jira/browse/JCR-4947 ....
Received response from https://issues.apache.org/jira/browse/JCR-4948 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4957 ....
Sending request to https://issues.apache.org/jira/browse/JCR-4958 ....
Received response from https://issues.apache.org/jira/browse/JCR-4950 ....
Received response from https://issues.apache.org/jira

title is:  [JCR-4430] fix javadoc:javadoc for jdk >= 11 - ASF JIRA
title is:  [JCR-4431] Release Jackrabbit 2.16.4 - ASF JIRA
title is:  [OAK-8295] Update version of jackrabbit dependency - ASF JIRA
title is:  [JCR-4433] Release Jackrabbit 2.19.3 - ASF JIRA
title is:  [JCR-4434] Download page gpg example needs second parameter - ASF JIRA
title is:  [JCR-4435] Archive Jackrabbit wiki - ASF JIRA
title is:  [OAK-8320] Improve tests for o.a.j.oak.security.authentication.token package - ASF JIRA
title is:  [JCR-4437] Update Tika dependency to 1.21 - ASF JIRA
title is:  [JCR-4438] jcr-webdav: AbstractLocatorFactory - fix Javadoc and add trace logging - ASF JIRA
title is:  [JCR-4439] disable baseline check for jackrabbit-bundle - ASF JIRA
title is:  [JCR-4440] remove jackrabbit-bundle - ASF JIRA
title is:  [JCR-4441] Update surefire/failsafe dependencies to 2.22.2 - ASF JIRA
title is:  [JCR-4442] Update animal-sniffer dependency to 1.18 - ASF JIRA
title is:  [JCR-4443] set baseline comparison

title is:  [JCR-4543] Update Tika dependency to 1.24 - ASF JIRA
title is:  [JCR-4544] Update slf4j dependency to 1.7.30 - ASF JIRA
title is:  [JCR-4545] Update httpclient/mime dependencies to 4.5.12 - ASF JIRA
title is:  [JCR-4546] Update aws java sdk version to 1.11.700 (consistent with Oak) - ASF JIRA
title is:  [JCR-4547] Update tomcat dependency to 8.5.53/7.0.103 - ASF JIRA
title is:  [JCR-4548] update javadoc-plugin dependency to 3.2.0 - ASF JIRA
title is:  [JCR-4549] backup with RepositoryCopier.copy() fails on second method call - ASF JIRA
title is:  [JCR-4550] Release Jackrabbit 2.16.6 - ASF JIRA
title is:  [JCR-4551] Use the normalized MediaType to check if the given MediaType should be indexed - ASF JIRA
title is:  [JCR-4552] EOL Jackrabbit 2.18 - ASF JIRA
title is:  [JCR-4553] release Jackrabbit 2.18.6 - ASF JIRA
title is:  [JCR-4554] Update Tika dependency to 1.24.1 - ASF JIRA
title is:  [JCR-4555] Update mockito dependency to 3.3.3 - ASF JIRA
title is:  [JCR-4556] Update t

title is:  [JCR-4653] Update httpclient/mime dependencies to 4.5.13 - ASF JIRA
title is:  [JCR-4654] webapp: update htmlunit dependency to 2.45.0 - ASF JIRA
title is:  [JCR-4655] Update mockito dependency to 3.6.28 - ASF JIRA
title is:  [JCR-4656] Upgrade Commons VFS to 2.7.0 - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log i

title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log in - ASF JIRA
title is:  Log